## GRID SEARCH for pressure

 
This code is used to find the best architecture for the NN for the pressure, we try with different number of hidden layers and different number of neurons in each layer. You can run the code for each architecture and then compare the results to find the best one.

This code was run in a local machine without a GPU.

Make sure the `data` folder is in the principal directory.



In [ ]:
# General setups and imports
from utils import *

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Set the seed for reproducibility
if device=="cuda:0":
    torch.cuda.empty_cache()
    torch.cuda.manual_seed(42)
    torch.cuda.init()
    torch.cuda.empty_cache()
    
np.random.seed(42)
torch.manual_seed(42)


In [ ]:
# Normalization of input parameters in [0,1]
maxs = np.array([8.0, 0.3, 0.5, 0.5, 0.5, 0.0])
mins = np.array([4.0, 0.1, -0.1, -0.5, -0.5, -0.3])
for i in range(params.shape[1]):
    params[:, i] = (params[:, i] - mins[i]) / (maxs[i] - mins[i])


# shuffle the parameters
idx = np.random.permutation(params.shape[0])
params = params[idx]

# Expand pressure in time
pressure_time = solutions['pressure'] @ basis_time['pressure'].T

# shuffle the pressure, the parameters are shuffled in the same way
pressure_time = pressure_time[idx]

# Split the data into training, validation and test set

# Training set: 80% of the data
# Validation set: 10% of the data
# Test set: 10% of the data

# Training set
params_train = params[:int(0.8 * len(params))]
pressure_time_train = pressure_time[:int(0.8 * len(params))]

# Validation set
params_val = params[int(0.8 * len(params)):int(0.9 * len(params))]
pressure_time_val = pressure_time[int(0.8 * len(params)):int(0.9 * len(params))]


# Test set
params_test = params[int(0.9 * len(params)):]
pressure_time_test = pressure_time[int(0.9 * len(params)):]


# Treat time as a parameter: add it to the parameter list
# u1 u2 u3 u4 u5 u6 t
times = np.linspace(0, 1, 300)

#sample all the times with times[:] 
#sample every 5 timesteps with times[::5] 
times_test= times[::5]
times_train= times[::5]
times_val= times[::5]

# generate a matrix of new parameters copying parameter vector for each time step for train, validation and test set
# Add the time as last parameter

params_time_train = np.repeat(params_train, len(times_train), axis=0)
params_time_train = np.hstack((params_time_train, np.tile(times_train, len(params_train)).reshape(-1, 1)))

params_time_val = np.repeat(params_val, len(times_val), axis=0)
params_time_val = np.hstack((params_time_val, np.tile(times_val, len(params_val)).reshape(-1, 1)))

params_time_test = np.repeat(params_test, len(times_test), axis=0)
params_time_test = np.hstack((params_time_test, np.tile(times_test, len(params_test)).reshape(-1, 1)))

# if times[:] put vel_time_test[:, :, :]
# if times[::5] put vel_time_test[:, :, ::5] 
pressure_model_test= pressure_time_test[:, :, ::5]
pressure_model_train= pressure_time_train[:, :, ::5]
pressure_model_val= pressure_time_val[:, :, ::5]

# Reshape the data to have the form (number of samples, number of parameters, number of time steps)
pressure_model_train = pressure_model_train.transpose(0, 2, 1).reshape((pressure_model_train.shape[0] * len(times_train)), 7)
pressure_model_val = pressure_model_val.transpose(0, 2, 1).reshape((pressure_model_val.shape[0] * len(times_val)), 7)
pressure_model_test = pressure_model_test.transpose(0, 2, 1).reshape((pressure_model_test.shape[0] * len(times_test)), 7)

# Normalize the SV coefficients of the pressure
sv_space_pressure = sv_space['pressure']
sv_space_pressure = sv_space_pressure / np.sum(sv_space_pressure)


# Convert to tensor
params_time_train = torch.tensor(params_time_train, dtype=torch.float32).to(device)
params_time_val = torch.tensor(params_time_val, dtype=torch.float32).to(device)
params_time_test = torch.tensor(params_time_test, dtype=torch.float32).to(device)

pressure_model_train = torch.tensor(pressure_model_train, dtype=torch.float32).to(device)
pressure_model_val = torch.tensor(pressure_model_val, dtype=torch.float32).to(device)
pressure_model_test = torch.tensor(pressure_model_test, dtype=torch.float32).to(device)

sv_space_pressure = torch.tensor(sv_space_pressure, dtype=torch.float32).to(device)
sv_space_pressure = sv_space_pressure.reshape(7, 1)





Choose the architecture of the network, run only the cell corresponding to the architecture you want to use



In [ ]:
# 2 hidden layers with K neurons each with batch normalization

class Net(torch.nn.Module):

    def __init__(self, input_size, hidden_size, output_size):
        super(Net, self).__init__()
        self.fc1 = torch.nn.Linear(input_size, hidden_size)
        self.F1 = torch.nn.Tanh()
        self.batch_norm1 = torch.nn.BatchNorm1d(hidden_size)
        self.fc2 = torch.nn.Linear(hidden_size, hidden_size)
        self.F2 = torch.nn.Tanh()
        self.fc3 = torch.nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = self.F1(self.fc1(x))
        x = self.batch_norm1(x)
        x = self.F2(self.fc2(x))
        return self.fc3(x)


In [ ]:
# 3 hidden layers with K neurons each with batch normalization

class Net(torch.nn.Module):

    def __init__(self, input_size, hidden_size, output_size):
        super(Net, self).__init__()
        self.fc1 = torch.nn.Linear(input_size, hidden_size)
        self.F1 = torch.nn.Tanh()
        self.batch_norm1 = torch.nn.BatchNorm1d(hidden_size)
        self.fc2 = torch.nn.Linear(hidden_size, hidden_size)
        self.F2 = torch.nn.Tanh()
        self.batch_norm2 = torch.nn.BatchNorm1d(hidden_size)
        self.fc3 = torch.nn.Linear(hidden_size, hidden_size)
        self.F3 = torch.nn.Tanh()
        self.fc4 = torch.nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = self.F1(self.fc1(x))
        x = self.batch_norm1(x)
        x = self.F2(self.fc2(x))
        x = self.batch_norm2(x)
        x = self.F3(self.fc3(x))
        return self.fc4(x)

In [ ]:
# 4 hidden layers with K neurons each with batch normalization

class Net(torch.nn.Module):

    def __init__(self, input_size, hidden_size, output_size):
        super(Net, self).__init__()
        self.fc1 = torch.nn.Linear(input_size, hidden_size)
        self.F1 = torch.nn.Tanh()
        self.batch_norm1 = torch.nn.BatchNorm1d(hidden_size)
        self.fc2 = torch.nn.Linear(hidden_size, hidden_size)
        self.F2 = torch.nn.Tanh()
        self.batch_norm2 = torch.nn.BatchNorm1d(hidden_size)
        self.fc3 = torch.nn.Linear(hidden_size, hidden_size)
        self.F3 = torch.nn.Tanh()
        self.batch_norm3 = torch.nn.BatchNorm1d(hidden_size)
        self.fc4 = torch.nn.Linear(hidden_size, output_size)
        self.F4 = torch.nn.Tanh()
        self.fc5 = torch.nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = self.F1(self.fc1(x))
        x = self.batch_norm1(x)
        x = self.F2(self.fc2(x))
        x = self.batch_norm2(x)
        x = self.F3(self.fc3(x))
        x = self.batch_norm3(x)
        x = self.F4(self.fc4(x))
        return self.fc5(x)

In [ ]:
# 5 hidden layers with K neurons each with batch normalization

class Net(torch.nn.Module):
    
    def __init__(self, input_size, hidden_size, output_size):
        super(Net, self).__init__()
        self.fc1 = torch.nn.Linear(input_size, hidden_size)
        self.F1 = torch.nn.ReLU()
        self.batch_norm1 = torch.nn.BatchNorm1d(hidden_size)
        self.fc2 = torch.nn.Linear(hidden_size, hidden_size)
        self.F2 = torch.nn.Tanh()
        self.batch_norm2 = torch.nn.BatchNorm1d(hidden_size)
        self.fc3 = torch.nn.Linear(hidden_size, hidden_size)
        self.F3 = torch.nn.ReLU()
        self.batch_norm3 = torch.nn.BatchNorm1d(hidden_size)
        self.fc4 = torch.nn.Linear(hidden_size, hidden_size)
        self.F4 = torch.nn.Tanh()
        self.batch_norm4 = torch.nn.BatchNorm1d(hidden_size)
        self.fc5 = torch.nn.Linear(hidden_size, hidden_size)
        self.F5 = torch.nn.ReLU()
        self.fc6 = torch.nn.Linear(hidden_size, output_size)


    def forward(self, x):
        x = self.F1(self.fc1(x))
        x = self.batch_norm1(x)
        x = self.F2(self.fc2(x))
        x = self.batch_norm2(x)
        x = self.F3(self.fc3(x))
        x = self.batch_norm3(x)
        x = self.F4(self.fc4(x))
        x = self.batch_norm4(x)
        x = self.F5(self.fc5(x))
        return self.fc6(x)

Change the value of the variable `hidden_size` to change the number of neurons in the hidden layer. 

The number of hidden layers is determined by the architecture of the network.


In [ ]:
# Dimension of the network

input_size = 7
# Choose the number of neurons in the hidden layer
hidden_size = 128 # Change the number of neurons in the hidden layer, we tried with 32, 64, 128, 256
output_size = 7 # POD coefficients for the pressure

# Create the network
net = Net(input_size=input_size, hidden_size=hidden_size, output_size=output_size).to(device)

# Loss function
loss_fn = torch.nn.MSELoss().to(device)

learning_rate = .01 # Starting learning rate

# Use the Adam optimizer
optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)


# Save the loss function for each iteration
losses_train = []
losses_val = []

# Save the absolute error for each iteration
err_val = []
err_train = []

Training of the network, the loss function is saved for each iteration. 
The network is trained for $1500$ epochs, you can change the number of epochs by changing the value of the variable `n_epochs`.

In [ ]:
n_epochs = 1500 # Number of epochs

for t in range(n_epochs):
    net.train()

    # Forward pass: compute predicted y by passing x to the model.
    y_pred = net(params_time_train).to(device)

    # Compute train loss
    loss_train = loss_fn(y_pred, pressure_model_train)

    losses_train.append(loss_train.item())

    # Before the backward pass, use the optimizer object to zero all of #the gradients for the variables it will update (which are the learnable
    # weights of the model)
    optimizer.zero_grad()

    # Backward pass: compute gradient of the loss with respect to model
    # parameters
    loss_train.backward()

    # Calling the step function on an Optimizer makes an update to its
    # parameters
    optimizer.step()
    
    # Validation
    net.eval()
    with torch.no_grad():
        y_pred_val = net(params_time_val).to(device)
        loss_val = loss_fn(y_pred_val, pressure_model_val)
        
    if t % 100 == 0:
        print("Epoch: ", t, "Train Loss: ", loss_train.item(),", Validation Loss: ", loss_val.item())
        
    losses_val.append(loss_val.item())

Test the network on the test set, the relative and absolute error are computed and printed.

In [ ]:
""" Test the network"""

# Evaluate the model on the test set
y_pred = net(params_time_test).to("cpu")

# convert the output of the NN to numpy array
y_pred_numpy = y_pred.detach().numpy()

# convert the pressure_model_test to numpy
pressure_model_test_numpy=pressure_model_test.to("cpu").detach().numpy()

# Compute and print loss.
loss_t = torch.nn.MSELoss()(y_pred, pressure_model_test.to("cpu"))

print("Test loss: ", loss_t.item())

# Compute the relative error

rel_error = np.linalg.norm(y_pred_numpy - pressure_model_test_numpy, axis=1) / np.linalg.norm(pressure_model_test_numpy, axis=1)

#Compute average values 
mean_pres=np.mean(np.linalg.norm(pressure_model_test_numpy, axis=1).reshape(-1, len(times_test)), axis=1)

# repeat the mean for every time step
mean_pres = np.repeat(mean_pres, len(times_test)).reshape(1, -1)

#Compute absolute error rescaled by average values within each simulation
abs_error = np.linalg.norm(y_pred_numpy - pressure_model_test_numpy, axis=1)/mean_pres
abs_error = abs_error.T

print("Relative error: ", np.mean(rel_error))
print("Absolute error: ", np.mean(abs_error))